# Problemstatement:  Kann regelmäßiges Renditeverhalten einer Aktie für das Jahr dargestellt werden?

In [ ]:
# nötige librarys importieren
import requests
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math 
import numpy as np

## Datenimport täglicher Handelsdaten einer Aktie über API

In [ ]:
# den API-key speichern, der von der Seite frei vergeben wird.
# der API-provider ist https://www.alphavantage.co/
apikey = ''

### Abruf des Search-Engine des API-providers, um den Code für das börsennotierte Unternehmen zu bekommen.

In [ ]:
# Eingabe des gesuchten börsennotierten Unternehmen
company = input("Please enter companys name ")

In [ ]:
# Hier werden die Parameter für den späteren request erstellt. 
params_1 = dict(apikey = apikey)
print(params_1)

In [ ]:
# den Search-URL per Stringoperationen bauen
search_url = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords='+company
response_search = requests.get(search_url, params =params_1)
response_search.url

In [ ]:
# Daten bzw. Suchergebnisse abrufen
response_search.json()
print(json.dumps(response_search.json(), indent=4))

### Datenabruf der täglichen Handelsdaten beim API-provider 

In [ ]:
# Eingabe des gesuchten Unternehmens mit dem Symbol per copy-paste
company_symbol = input('Please enter the companys symbol ')

In [ ]:
# Hier werden die Parameter für den späteren request erstellt:
# 'full', da nach Angaben des API-providers per default 'compact' 100 Tage abgerufen würden und mit 'full' alle vorhandenen.
params_2 = dict(outputsize= 'full', apikey = apikey)
print(params_2)

In [ ]:
# URL per Stringoperationen bauen
url_data = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+company_symbol
response_data = requests.get(url_data, params = params_2)
response_data.url

In [ ]:
# Daten bzw. Handelsdaten abrufen und der Variable 'data' zuordnen
data = response_data.json()
print(json.dumps(response_data.json(), indent=4))

In [ ]:
# dictionary-schüssel ermitteln für weitere Verarbeitung
response_data.json().keys()

In [ ]:
# Data clean: Metadaten brauchen wir nicht
# dictionary-schüssel zweiter Ebene ist das Datum des Tages
dates = data['Time Series (Daily)'].keys()
dates

In [ ]:
# Aus dem dict machen wir ein pd.DataFrame
companys_daily = pd.DataFrame(data['Time Series (Daily)']).T
print(companys_daily.head())
print(companys_daily.shape)

In [ ]:
# Die Datentypen müssen für die Verarbeitung passen
companys_daily.dtypes

In [ ]:
# Für die Analyse werden hier nur die Spalte close und open gebraucht, um einen stetigen Verlauf zu gewährleisten,
# da die beiden Spalten Preise darstellen passt der Datentyp float
companys_daily=companys_daily.astype(float)
companys_daily.dtypes

In [ ]:
companys_daily.sort_index(ascending=True, inplace=True)
companys_daily

In [ ]:
# Tagesrendite = Schlusskurs - Eröffnungskurs und, da wir als Ziel haben die Jahre miteinander zu vergleichen, 
# bietet sich als Vergleichseinheit % an
companys_daily.loc[:,'daily_return'] = 0
for i in range(0,companys_daily.shape[0]):
    if i == 0:
        companys_daily.iloc[i,8] = (companys_daily.iloc[i,3]-companys_daily.iloc[i,0])/companys_daily.iloc[i,0]*100
    else:
        companys_daily.iloc[i,8] = (companys_daily.iloc[i,3]-companys_daily.iloc[i-1,3])/companys_daily.iloc[i-1,3]*100

In [ ]:
companys_daily.sort_index(ascending=False, inplace=True)
companys_daily

In [ ]:
# wie viele fehlerhafte Einträge gibt es?
companys_daily['daily_return'].isnull().sum()

In [ ]:
# Die rohen Daten sind nun vorhanden
print(companys_daily)
print(companys_daily.shape)

## Erstellung vom DataFrame täglicher Rendite

In [ ]:
# für die Analyse brauchen wir nur die Tagesrenditen
companys_daily_return=companys_daily['daily_return']
companys_daily_return.head()

In [ ]:
# Eine erste Visualisierung zu anschauungszwecken
fig = px.line(companys_daily_return,y='daily_return',title=company)
fig.show()

In [ ]:
# wie viele Handelstage haben wir im Datenimport?
companys_daily_return.shape[0]

In [ ]:
# Da als nächstes die Datentransformierungen beginnen, erstmal die Daten durch eine Kopie absichern
companys_daily_return = pd.DataFrame(companys_daily_return)
companys_daily_return2 = companys_daily_return.copy()
companys_daily_return2.head()

In [ ]:
# Abfrage vom Speicher_pfad zum Speicherordner vom User-Pc
pfad = input('Gebe den Pfad zum Ordner hier ein ')


## Analyse gleichgewichteter wöchentlicher Rendite 

### Dataset Slicing

In [ ]:
# Um den Arbeitspeicher zu entlassten, slicen wir für die Analyse irrelavante Jahre weg und damit irrelevante Daten.
# Festlegung vom abgeschlossenen Zeitintervall, auf dem die Rendite analysiert wird.
date_end = input('Tippe die Jahrzahl ein, welche den Zeitraum von links begrenzt (empf. 3-5 Jahre Abstand zwischen den Jahreszahlen): ')
date_end_titel = date_end
date_end= int(date_end)
date_start = input('Tippe die Jahrzahl ein, welche den Zeitraum von rechts begrenzt (empf. 3-5 Jahre Abstand zwischen den Jahreszahlen): ')
date_start_titel = date_start
date_start= int(date_start)

In [ ]:
# Import der Funktion 'date', um das Datum aus dem Index bearbeiten zu können.
from datetime import date

In [ ]:
# Auf welchem Zeitraum werden wir slicen?
[str(date(date_end,1,1)),str(date(date_start,12,31))]


In [ ]:
# jetzt slicen wir auf das Zeitintervall
companys_daily_return_glge = companys_daily_return2.loc[str(date(date_start,12,31)):str(date(date_end,1,1)),:].copy()
companys_daily_return_glge

### Bestimmung täglicher gleichgewichteter Rendite

In [ ]:
# zur Bearbeitung das Datum in eine Spalte aus dem Index extrahieren 
companys_daily_return_glge.reset_index(inplace=True)
companys_daily_return_glge.head()

In [ ]:
# der Datentyp wird zu Datum-Format angepasst
companys_daily_return_glge.loc[:,'index']=pd.to_datetime(companys_daily_return_glge.loc[:,'index'])
companys_daily_return_glge.dtypes

### Bestimmung jahresübergreifender wöchentlicher Rendite

In [ ]:
# Jedem Datensatz die KW zuordnen
companys_daily_return_glge.loc[:,'Year']= companys_daily_return_glge.loc[:,'index'].dt.isocalendar().year
companys_daily_return_glge.loc[:,'index']= companys_daily_return_glge.loc[:,'index'].dt.isocalendar().week
companys_daily_return_glge.head()

In [ ]:
# Dies ist nur ein  Testdurchlauf, um nachzumessen, ob der Renditeverfall zum Coronatief 2020 übereinstimmt.
# setze hierfür das Zeitintervall auf 2020 bis 2020
a = np.concatenate((companys_daily_return_glge.loc[companys_daily_return_glge.loc[:,'index']==7,'daily_return']/100+1,companys_daily_return_glge.loc[companys_daily_return_glge.loc[:,'index']==8,'daily_return']/100+1))
b = np.concatenate((a,companys_daily_return_glge.loc[companys_daily_return_glge.loc[:,'index']==9,'daily_return']/100+1))
c = np.concatenate((b,companys_daily_return_glge.loc[companys_daily_return_glge.loc[:,'index']==10,'daily_return']/100+1))
d = np.concatenate((c,companys_daily_return_glge.loc[companys_daily_return_glge.loc[:,'index']==11,'daily_return']/100+1))
d.prod()

In [ ]:
# wie große ist das Dataset nun?
companys_daily_return_glge.shape

In [ ]:
# gibt es Kalenderwochen die in unserem Data set fehlen und wenn ja, welche?
null_week = list(range(0,53-companys_daily_return_glge['index'].unique().shape[0]))
j = 0
for i in range(1,54):
    if i not in companys_daily_return_glge['index'].unique():
        null_week[j] = i
        j +=1
null_week

In [ ]:
# Formatierung zum DataFrame
null_weeks_and_return = pd.DataFrame([null_week,[0 for k in range(len(null_week))], [0 for k in range(len(null_week))]]).T
null_weeks_and_return

In [ ]:
# Die fehlenden Kalenderwochen werden mit Rendite 0 hinzugefügt
companys_daily_return_glge = pd.DataFrame(np.vstack((companys_daily_return_glge,null_weeks_and_return)))
companys_daily_return_glge.columns = ['index','daily_return','Year']
companys_daily_return_glge.shape

In [ ]:
# die KW53 ist unregelmäßig, daher wird es sinn ergeben, 
# die Häufigkeit der Wochenrenditen im dataset, um die letzte KW des Jahres zu speichern, vor dem Aggregieren nach KW
A_KWmax = companys_daily_return_glge.loc[companys_daily_return_glge.loc[:,'index'] == companys_daily_return_glge['index'].max(),'index'].count()
A_KW52 = companys_daily_return_glge.loc[companys_daily_return_glge.loc[:,'index'] == 52,'index'].count()
A_KW1 = companys_daily_return_glge.loc[companys_daily_return_glge.loc[:,'index'] == 1,'index'].count()

In [ ]:
# Alle Datensätze einer KW und eines Jahres werden gruppiert und der daily_return durch Produktbildung aggregiert.
companys_daily_return_glge['daily_return'] = companys_daily_return_glge['daily_return']/100 + 1
companys_daily_return_glge = companys_daily_return_glge.groupby(['index', 'Year']).prod().reset_index()
companys_daily_return_glge['daily_return'] = (companys_daily_return_glge['daily_return'] - 1)*100
companys_daily_return_glge.head()

In [ ]:
# das Ergebnis geteilt durch die Anzahl der Jahre.
companys_daily_return_glge = companys_daily_return_glge.drop(['Year'], axis=1)
companys_daily_return_glge = companys_daily_return_glge.groupby(['index']).mean().reset_index()
companys_daily_return_glge.head()

In [ ]:
# den wahren Index und die Kategorienspalte umbennen
companys_daily_return_glge.columns = ['Week','weekly_return']
companys_daily_return_glge.head()

In [ ]:
# Die KW wird zum Index
companys_daily_return_glge.set_index('Week', inplace = True)
companys_daily_return_glge.head()

### Visualisierung wöchentlicher Rendite gleichgewichtet

#### Bildung eines dreiwöchigen Mittelwerts zur Darstellung von Zeitabschnitten positiven und negativen Renditeverhaltens

In [ ]:
# Für den Fall, dass der Zeitraum so gewählt ist, dass es keine KW53 gibt, wird die letzte KW in einer Variable gespeichert:
last_week = len(companys_daily_return_glge['weekly_return'])
last_week

In [ ]:
# Erstellung einer neuen Spalte mit dem dreiwöchigen Mittelwert im DataFrame vereinfacht das iterieren
companys_daily_return_glge['drei_Wochen_Mittelwert']=0
# Der Loop läuft nun über die Länge des DataFrames:
for i in range(1,last_week+1):
    # KW1 hat uneindeutige Zuordnung zur vorherigen KW, und zwar KW52 und KW53.
    # In KW1 nimmt der Mittelwert die KW53*#KW53/(#KW52+#KW53) und KW52*#KW52/(#KW52+#KW53), 
    # somit werden KW52 und KW53 so gewichtet, dass sie dem Wert nach nur einer KW entsprechen,
    # um das Problem der unregelmäßigen KW53 zu überwinden.
    if i==1:
        companys_daily_return_glge.loc[i,'drei_Wochen_Mittelwert'] = (companys_daily_return_glge.loc[52,'weekly_return']*A_KW52/(A_KW52+A_KWmax) + 
                                                                      companys_daily_return_glge.loc[last_week,'weekly_return']*A_KWmax/(A_KW52+A_KWmax) + 
                                                                      companys_daily_return_glge.loc[1,'weekly_return'] + 
                                                                      companys_daily_return_glge.loc[2,'weekly_return'])/3
    
    # KW52 hat uneindeutige Zuordnung zur nächsten KW, und zwar KW53 und KW1.
    # In KW52 nimmt der Mittelwert die KW53*#KW53/(#KW1+#KW53) und KW1*#KW1/(#KW1+#KW53), 
    # somit werden KW1 und KW53 so gewichtet, dass sie dem Wert nach nur einer KW entsprechen 
    elif i==52:
        companys_daily_return_glge.loc[i,'drei_Wochen_Mittelwert'] = (companys_daily_return_glge.loc[51,'weekly_return'] +  
                                                                      companys_daily_return_glge.loc[52,'weekly_return'] + 
                                                                      companys_daily_return_glge.loc[last_week,'weekly_return']*A_KWmax/(A_KW1+A_KWmax) + 
                                                                      companys_daily_return_glge.loc[1,'weekly_return']*A_KW1/(A_KW1+A_KWmax))/3
    
    # die KW 53 hat eindeutige Zuordnung zur vorherigen und nächsten KW, 
    # ledigleich die nächste KW kann nicht im loop bestimmt werden.
    elif i==53:   
        companys_daily_return_glge.loc[53,'drei_Wochen_Mittelwert'] = (companys_daily_return_glge.loc[52,'weekly_return'] + 
                                                                      companys_daily_return_glge.loc[53,'weekly_return'] + 
                                                                      companys_daily_return_glge.loc[1,'weekly_return'])/3
    
    # sonst bildet der Loop pro Datensatz den Mittelwert von der Woche davor, von der momentanen und von der danach
    else:
        companys_daily_return_glge.loc[i,'drei_Wochen_Mittelwert'] = (companys_daily_return_glge.loc[i-1,'weekly_return'] + 
                                                                      companys_daily_return_glge.loc[i,'weekly_return'] + 
                                                                      companys_daily_return_glge.loc[i+1,'weekly_return'])/3

In [ ]:
companys_daily_return_glge['acc_Rendite']=0

In [ ]:
j=0
# der for-loops muss zweimal laufen, um alle akkumulierten Werten hernaziehen zu können: wegen KW1.
while j<2:
    # Erstellung einer neuen Spalte im DataFrame vereinfacht das iterieren
    # Der Loop läuft nun über die Länge des DataFrames:
    for i in range(1,last_week+1):
        # sollte der Mittelwert zwischen dieser Woche und der nächsten und der letzten das Vorzeichen beibehalten,
        # dann soll der Loop weiter Gewinn bzw. Verlust accumulieren, allerdings nach den Wochenrenditen, nicht nach dem Mittelwert.
        # In KW1: nimmt der Loop die KW52 und die letzte KW wegen der unregelmäßigen KW53
        
        if i == 1 and np.sign(companys_daily_return_glge.loc[52,'weekly_return']*A_KW52/(A_KW52+A_KWmax)+companys_daily_return_glge.loc[last_week,'weekly_return']*A_KWmax/(A_KW52+A_KWmax)) == np.sign(companys_daily_return_glge.loc[1,'drei_Wochen_Mittelwert']) == np.sign(companys_daily_return_glge.loc[2,'drei_Wochen_Mittelwert']):
            if companys_daily_return_glge.loc[52,'acc_Rendite']*A_KW52/(A_KW52+A_KWmax) + companys_daily_return_glge.loc[last_week,'acc_Rendite']*A_KWmax/(A_KW52+A_KWmax) == 0:
                companys_daily_return_glge.loc[1,'acc_Rendite'] = 0
            else:
                companys_daily_return_glge.loc[1,'acc_Rendite'] = ((1+(companys_daily_return_glge.loc[52,'acc_Rendite']*A_KW52/(A_KW52+A_KWmax) + companys_daily_return_glge.loc[last_week,'acc_Rendite']*A_KWmax/(A_KW52+A_KWmax))/100)*(1 + companys_daily_return_glge.loc[1,'weekly_return']/100)-1)*100
        
        elif i == last_week and np.sign(companys_daily_return_glge.loc[last_week-1,'weekly_return']) == np.sign(companys_daily_return_glge.loc[last_week,'drei_Wochen_Mittelwert']) == np.sign(companys_daily_return_glge.loc[1,'drei_Wochen_Mittelwert']):
            if companys_daily_return_glge.loc[last_week-1,'acc_Rendite'] == 0:
                companys_daily_return_glge.loc[last_week,'acc_Rendite'] = 0
            else:
                companys_daily_return_glge.loc[last_week,'acc_Rendite'] = ((1+companys_daily_return_glge.loc[last_week-1,'acc_Rendite']/100) * (1+ companys_daily_return_glge.loc[last_week,'weekly_return']/100)-1)*100
        elif i!=1 and i!=last_week and np.sign(companys_daily_return_glge.loc[i,'drei_Wochen_Mittelwert']) == np.sign(companys_daily_return_glge.loc[i-1,'drei_Wochen_Mittelwert']) == np.sign(companys_daily_return_glge.loc[i+1,'drei_Wochen_Mittelwert']):
            if companys_daily_return_glge.loc[i-1,'acc_Rendite'] == 0:
                companys_daily_return_glge.loc[i,'acc_Rendite'] = companys_daily_return_glge.loc[i,'weekly_return']
            else:
                companys_daily_return_glge.loc[i,'acc_Rendite'] =  ((1+companys_daily_return_glge.loc[i-1,'acc_Rendite']/100) *(1+ companys_daily_return_glge.loc[i,'weekly_return']/100)-1)*100
        
        # sollte der Mittelwert nächste Woche von Gewinn zu Verlust oder umgekehrt wechseln,
        # dann soll der Loop die Akkumulierung neu bei Null beginnen, um klar Gewinn bzw. Verlustzeiträume abzugrenzen.
        else:
            companys_daily_return_glge.loc[i,'acc_Rendite'] = 0
    j+=1
companys_daily_return_glge

#### Visualisierung mittels Linien- und Flächendiagramm

In [ ]:
# Leere Figure erstellen
fig1 = go.Figure()
# einen Trace der chronologisch gewichteten wöchentlichen Rendite als Liniendiagramm hinzufügen
fig1.add_trace(go.Scatter(x=companys_daily_return_glge.index,y=companys_daily_return_glge.loc[:,'weekly_return'],
    name="weekly average return  <br>(indicator for volatility)"))
# einen Trace des dreitägigen Mittelwerts hinzufügen
fig1.add_trace(go.Scatter(x=companys_daily_return_glge.index,y=companys_daily_return_glge.loc[:,'drei_Wochen_Mittelwert'],fill='tozeroy',
    name="three weeks average <br>(indicator for +/- return periods)"))
# Zur Renditeeinschätzung von Zeitperioden die in denselben akkumulierte wöchentliche Rendite
fig1.add_trace(go.Scatter(x=companys_daily_return_glge.index,y=companys_daily_return_glge.loc[:,'acc_Rendite'],fill='tozeroy',
    name="accumulated return on period <br>(indicator for orders)"))
# Das Layout anpassen
fig1.update_layout(
    title=company +' equal weighted Analysis ['+ date_end_titel +' - '+date_start_titel+'] from '+str(date.today())+ ' ',
    xaxis_title="calendar week",
    yaxis_title="% return",
    font=dict(size=10,color="RebeccaPurple"),
    legend=dict(orientation="h", yanchor="bottom", y=1.02,
                xanchor="right",x=1))
# Die Abstände der Wochen auf der Anzeige der x-Achse für bessere Lesbarkeit verkleinern
fig1.update_xaxes(nticks=30)
fig1.show()

#### Export des Graphen als interaktive .html

In [ ]:
file1 = company+'_return_analysis_'+date_end_titel +'-'+date_start_titel

In [ ]:
fig1.write_html(pfad+'/'+file1 +'.html')

## Analyse wöchentlicher Rendite unter chronologisch abnehmender Gewichtung

### Dataset Slicing

In [ ]:
# Um den Arbeitspeicher zu entlassten, slicen wir für die Analyse irrelavante Jahre weg und damit irrelevante Daten.
# Festlegung vom abgeschlossenen Zeitintervall, auf dem die Rendite analysiert wird.
jahre = input('Tippe die Jahresanzahl (empf. 12) für die Analyse wöchentlicher Rendite unter chronologisch fallender Gewichtung ein: ')
jahre=int(jahre)

In [ ]:
# Import der Funktion 'date', um das Datum aus dem Index bearbeiten zu können.
from datetime import date

In [ ]:
# ab wann rückwirkend slicen wir?
date.today().year-jahre

In [ ]:
# wir definieren bzw. slicen unser Dataset also von heute bis zum Datum vor 'jahre'
companys_daily_return = companys_daily_return[str(date.today()):str(date(date.today().year-jahre,date.today().month,date.today().day))]
companys_daily_return.shape

### Abnehmende Gewichtung täglicher Rendite nach Jahr

In [ ]:
# Absicherung der Norm von daily_return bzw. der Abstand der einzelnen Werte von der Null
# Dies ist wichtig, da nach der Gewichtung der Abstand zur Null  wieder hergestellt werden muss - im Sinne eines Mittelwertes
abs_total = abs(companys_daily_return.loc[:,'daily_return']).sum()
abs_total

In [ ]:
# zur Bearbeitung das Datum in eine Spalte aus dem Index extrahieren 
companys_daily_return.reset_index(inplace=True)
companys_daily_return.head()

In [ ]:
# der Datentyp wird zu Datum-Format angepasst
companys_daily_return['index']=pd.to_datetime(companys_daily_return['index'])
companys_daily_return.dtypes

In [ ]:
# Die Datensätze werden mit dem zugehörigen Jahr gekennzeichnet
companys_daily_return['Year']= companys_daily_return['index'].dt.isocalendar().year
companys_daily_return.head()

In [ ]:
# Kontrolle: Jahre die im Dataset sind
years = companys_daily_return['Year'].unique()
years

In [ ]:
companys_daily_return_EMA = companys_daily_return.copy()

In [ ]:
# Bestimmung des Faktors für die Gewichtung der Daten nach exponentiellem Zerfall. 
# Ein Schrumpfaktor ergibt mehr Sinn als linearer Zerfall inspiriert am "Exponential Smoothing"
# unter der Annahme, dass s_0 = (1-alpha)**t * x_0, wobei x_0 die Werte der KWs des ältesten jahres (nach 'jahre') sind.
# (1-alpha)='rate', damit ist alpha in [0,1] klein nach Empfehlung zur Abschätzung von unten beim smoothing factor und Input.

rate = round(0.1**(1/jahre),3)
rate

In [ ]:
# Die Gewichtung wird mit einem Loop bestimmt, über die Länge des DataFrames:
# anders als beim "Exponential Smoothing" wird hier keine Abschätzung von unten verfolgt, sondern eine Einschätzung.
# Daher wird x_t also die Beobachtung aus dem aktuellen Jahr und dem davor nicht mit alpha verkleinert, sondern beibehalten.
# dadurch erübrig sich auch die Problamik von der Relevanz von x_0 beim "Exponential Smoothing".
# Diesbezüglich ist allerdings eine konventionelle abschätzung von oben nötig, und zwar infolge die EMA(Jahre).
for i in range(len(companys_daily_return['Year'])):
    # Sollte das Jahr in der Zeile i nicht das jetzige und nicht das letzte Jahr sein, dann wird gewichtet:
   if years.max()-1-companys_daily_return.loc[i,'Year']>0:
        # die tägliche Rendite mir der rate**(leztes_Jahr-Jahr_aus_Zeile_i).
        companys_daily_return.loc[i,'daily_return'] = companys_daily_return.loc[i,'daily_return']*rate**(years.max()-1-companys_daily_return.loc[i,'Year'])

In [ ]:
# Da das jetzige Jahr die Gewichtung 1 hat ist offensichtlich, dass die nun gewichteten Werte größer sind als ursprünglich.
# Entsprechend müssen alle Werte normiert werden, damit eine realistische Einschätzung künftiger Rendite pro Woche entsteht. 
ge_abs_total = abs(companys_daily_return.loc[:,'daily_return']).sum()
ge_abs_total

In [ ]:
# Bestimmung des Faktors zur Widerherstellung des Abstandes zur Null - Normierungsfaktor:
faktor_norm = abs_total/ge_abs_total
# Es hat sich bewährt diese Berechnung etwas anzupassen, da die EMA deutlich größere Wert ausgibt:
faktor_norm = faktor_norm *1.8
faktor_norm

In [ ]:
# Wiederherstellung des Abstandes zu Null der nun gewichteten Werte - Normierung:
companys_daily_return.loc[:,'daily_return'] = companys_daily_return.loc[:,'daily_return']*faktor_norm
companys_daily_return.head()

### Bestimmung jahresübergreifender wöchentlicher Rendite

In [ ]:
# Zuorndung von jedem Datensatz zu seiner KW
companys_daily_return['index']= companys_daily_return['index'].dt.isocalendar().week
companys_daily_return.head()

In [ ]:
# wie große ist das Dataset nun?
companys_daily_return.shape

In [ ]:
# gibt es Kalenderwochen die in unserem Data set fehlen und wenn ja, welche?
null_week2 = list(range(0,53-companys_daily_return['index'].unique().shape[0]))
j = 0
for i in range(1,54):
    if i not in companys_daily_return['index'].unique():
        null_week2[j] = i
        j +=1
null_week2

In [ ]:
# Formatierung zum DataFrame
null_weeks_and_return2 = pd.DataFrame([null_week2,[0 for k in range(len(null_week2))],[0 for k in range(len(null_week2))]]).T
null_weeks_and_return2

In [ ]:
# Die fehlenden Kalenderwochen werden mit Rendite 0 hinzugefügt
companys_daily_return = pd.DataFrame(np.vstack((companys_daily_return,null_weeks_and_return2)))
companys_daily_return.columns = ['index','daily_return','Year']
companys_daily_return.shape

In [ ]:
# die KW53 ist unregelmäßig, daher wird es sinn ergeben, 
# die Häufigkeit der Wochenrenditen im dataset, um die letzte KW des Jahres zu speichern, vor dem Aggregieren nach KW
A_KWmax_2 = companys_daily_return.loc[companys_daily_return.loc[:,'index'] == companys_daily_return['index'].max(),'index'].count()
A_KW52_2 = companys_daily_return.loc[companys_daily_return.loc[:,'index'] == 52,'index'].count()
A_KW1_2 = companys_daily_return.loc[companys_daily_return.loc[:,'index'] == 1,'index'].count()

In [ ]:
companys_daily_return.head()

In [ ]:
# Alle Datensätze einer KW gruppieren durch arithmetischen Mittel aggregieren,
companys_daily_return = companys_daily_return.groupby('index').mean().reset_index()
companys_daily_return.head()

In [ ]:
# Die Spalte mit dem Jahr wird nicht mehr gebraucht
companys_daily_return = companys_daily_return.drop(['Year'], axis=1)
companys_daily_return.head()

In [ ]:
# und den Spaltennamen ändern
companys_daily_return.columns = ['Week','weekly_return']
companys_daily_return.head()

In [ ]:
# Die KW wird Index
companys_daily_return.set_index('Week', inplace = True)
companys_daily_return.head()

#### Berechnung der EMA zu den angegebenen Jahren zur Gewichtungskontrolle

In [ ]:
companys_daily_return_EMA['index']= companys_daily_return_EMA['index'].dt.isocalendar().week
companys_daily_return_EMA.head()

In [ ]:
# Alle Datensätze einer KW und eines Jahres werden gruppiert und der daily_return durch Produktbildung aggregiert.
companys_daily_return_EMA['daily_return'] = companys_daily_return_EMA['daily_return']/100 + 1
companys_daily_return_EMA = companys_daily_return_EMA.groupby(['index', 'Year']).prod().reset_index()
companys_daily_return_EMA['daily_return'] = (companys_daily_return_EMA['daily_return'] - 1)*100
companys_daily_return_EMA_mthd = companys_daily_return_EMA.copy()
companys_daily_return_EMA.head()

##### Berechnung der EMA mit Formel

In [ ]:
# EMA(jahre), EMA=Price(t)×k+EMA(y)×(1−k)where:t=todayy=yesterdayN=number of years in EMA_k=2÷(N+1)​
EMA_years = list(range(0,companys_daily_return_EMA.loc[:,'index'].max()))
# j läuft die Kalenderwochen ab.
for j in range(0,companys_daily_return_EMA.loc[:,'index'].max()):
    # Anzahl der Jahre für die Datensätze einer Kalenderwoche vorhanden sind 
    EMA_year = list(range(0,companys_daily_return_EMA.loc[companys_daily_return_EMA['index']==j+1,'index'].count()+1))
    # i läuft die Jahre einer Kalenderwoche ab.
    for i in range(0,companys_daily_return_EMA.loc[companys_daily_return_EMA['index']==j+1,'index'].count()):
        EMA_year[i+1] = companys_daily_return_EMA.loc[companys_daily_return_EMA['index']==j+1,:].iloc[i,2]*(2/(1+jahre)) + EMA_year[i]*(1-(2/(1+jahre)))
        
    EMA_years[j] = EMA_year[companys_daily_return_EMA.loc[companys_daily_return_EMA['index']==j+1,'index'].count()]

In [ ]:
EMA_year

In [ ]:
pd.array(EMA_years)

##### Berechnung der EMA mit der Methode .ewm

In [ ]:
companys_daily_return_EMA_mthd.head()

In [ ]:
# EMA(jahre), kann auch mit der Methode .ewm calculiert werden, spann = jahre:
# Liste der KW
EMA_KW_mthd = list(range(1,companys_daily_return_EMA_mthd.loc[:,'index'].max()+1))
# j läuft die Kalenderwochen ab.
for j in range(1,companys_daily_return_EMA_mthd.loc[:,'index'].max()+1):
    # Liste der Jahre für die Datensätze einer Kalenderwoche vorhanden sind 
    EMA_years_mthd = list(range(1,companys_daily_return_EMA_mthd.loc[companys_daily_return_EMA_mthd['index']==j,'index'].count()+1))
    # Jahre einer Kalenderwoche ab.
    EMA_KW_mthd[j-1] = list(companys_daily_return_EMA_mthd.loc[companys_daily_return_EMA_mthd['index']==j,:].iloc[:,2].ewm(span = len(EMA_years_mthd), adjust = False).mean())[-1]

In [ ]:
 pd.array(EMA_KW_mthd)

##### Vergleich der EMA-Berechnung

In [ ]:
EMA_compare = pd.DataFrame()
EMA_compare['EMA_year'] = pd.array(EMA_years)
EMA_compare['EMA_year_method'] = pd.array(EMA_KW_mthd)
EMA_compare.head()

In [ ]:
# Vergleich beider EMA-Berechnungen
# Leere Figure erstellen
fig1 = go.Figure()
# einen Trace der chronologisch gewichteten wöchentlichen Rendite als Liniendiagramm hinzufügen
fig1.add_trace(go.Scatter(x=EMA_compare.index,y=EMA_compare['EMA_year'], name="EMA_year"))
# einen Trace des dreitägigen Mittelwerts hinzufügen
fig1.add_trace(go.Scatter(x=EMA_compare.index,y=EMA_compare['EMA_year_method'], name="EMA_year_method"))
# Das Layout anpassen
fig1.update_layout(
    title='EMA Vergleich',
    xaxis_title="calendar week",
    yaxis_title="% return",
    font=dict(size=10,color="RebeccaPurple"),
    legend=dict(orientation="h", yanchor="bottom", y=1.02,
                xanchor="right",x=1))
# Die Abstände der Wochen auf der Anzeige der x-Achse für bessere Lesbarkeit verkleinern
fig1.update_xaxes(nticks=30)
fig1.show()

### Visualisierung wöchentlicher Rendite unter chronologisch abnehmender Gewichtung

#### Bildung eines dreiwöchigen Mittelwerts zur Darstellung von Zeitabschnitten positiven und negativen Renditeverhaltens

In [ ]:
# Für den Fall, dass der Zeitraum so gewählt ist, dass es keine KW53 gibt, wird die letzte KW in einer Variable gespeichert:
last_week2 = len(companys_daily_return['weekly_return'])
last_week2

In [ ]:
# Erstellung einer neuen Spalte mit dem dreiwöchigen Mittelwert im DataFrame vereinfacht das iterieren
# die letzte kalenderwoche im Dataset, KW1, KW52 und KW53 sind Sonderfälle
companys_daily_return['drei_Wochen_Mittelwert']=0
# Der Loop läuft nun über die Länge des DataFrames:
for i in range(1,last_week2+1):
    # KW1 hat uneindeutige Zuordnung zur vorherigen KW, und zwar KW52 und KW53.
    # In KW1 nimmt der Mittelwert die KW53*#KW53/(#KW52+#KW53) und KW52*#KW52/(#KW52+#KW53), 
    # somit werden KW52 und KW53 so gewichtet, dass sie dem Wert nach nur einer KW entsprechen,
    # um das Problem der unregelmäßigen KW53 zu überwinden.
    if i==1:
        companys_daily_return.loc[i,'drei_Wochen_Mittelwert'] = (companys_daily_return.loc[52,'weekly_return']*A_KW52_2/(A_KW52_2+A_KWmax_2) + 
                                                                      companys_daily_return.loc[last_week2,'weekly_return']*A_KWmax_2/(A_KW52_2+A_KWmax_2) + 
                                                                      companys_daily_return.loc[1,'weekly_return'] + 
                                                                      companys_daily_return.loc[2,'weekly_return'])/3
    
    # KW52 hat uneindeutige Zuordnung zur nächsten KW, und zwar KW53 und KW1.
    # In KW52 nimmt der Mittelwert die KW53*#KW53/(#KW1+#KW53) und KW1*#KW1/(#KW1+#KW53), 
    # somit werden KW1 und KW53 so gewichtet, dass sie dem Wert nach nur einer KW entsprechen 
    elif i==52:
        companys_daily_return.loc[i,'drei_Wochen_Mittelwert'] = (companys_daily_return.loc[51,'weekly_return'] +  
                                                                    companys_daily_return.loc[52,'weekly_return'] + 
                                                                    companys_daily_return.loc[last_week2,'weekly_return']*A_KWmax_2/(A_KW1_2+A_KWmax_2) + 
                                                                    companys_daily_return.loc[1,'weekly_return']*A_KW1_2/(A_KW1_2+A_KWmax_2))/3
    
    # die KW 53 hat eindeutige Zuordnung zur vorherigen und nächsten KW, 
    # ledigleich die nächste KW kann nicht im loop bestimmt werden.
    elif i==53:   
        companys_daily_return.loc[53,'drei_Wochen_Mittelwert'] = (companys_daily_return.loc[52,'weekly_return'] + 
                                                                    companys_daily_return.loc[53,'weekly_return'] + 
                                                                    companys_daily_return.loc[1,'weekly_return'])/3
    
    # sonst bildet der Loop pro Datensatz den Mittelwert von der Woche davor, von der momentanen und von der danach
    else:
        companys_daily_return.loc[i,'drei_Wochen_Mittelwert'] = (companys_daily_return.loc[i-1,'weekly_return'] + 
                                                                      companys_daily_return.loc[i,'weekly_return'] + 
                                                                      companys_daily_return.loc[i+1,'weekly_return'])/3

In [ ]:
companys_daily_return['acc_Rendite']=0

In [ ]:
j=0
# der for-loops muss zweimal laufen, um alle akkumulierten Werten hernaziehen zu können: wegen KW1.
while j<2:
    # Erstellung einer neuen Spalte im DataFrame vereinfacht das iterieren
    # Der Loop läuft nun über die Länge des DataFrames:
    for i in range(1,last_week2+1):
        # sollte der Mittelwert zwischen dieser Woche und der nächsten und der letzten das Vorzeichen beibehalten,
        # dann soll der Loop weiter Gewinn bzw. Verlust accumulieren, allerdings nach den Wochenrenditen, nicht nach dem Mittelwert.
        # In KW1: nimmt der Loop die KW52 und die letzte KW wegen der unregelmäßigen KW53
        
        if i == 1 and np.sign(companys_daily_return.loc[52,'weekly_return']*A_KW52_2/(A_KW52_2+A_KWmax_2)+companys_daily_return.loc[last_week2,'weekly_return']*A_KWmax_2/(A_KW52_2+A_KWmax_2)) == np.sign(companys_daily_return.loc[1,'drei_Wochen_Mittelwert']) == np.sign(companys_daily_return.loc[2,'drei_Wochen_Mittelwert']):
            if companys_daily_return.loc[52,'acc_Rendite']*A_KW52_2/(A_KW52_2+A_KWmax_2) + companys_daily_return.loc[last_week2,'acc_Rendite']*A_KWmax_2/(A_KW52_2+A_KWmax_2) == 0:
                companys_daily_return.loc[1,'acc_Rendite'] = companys_daily_return.loc[1,'weekly_return']
            else:
                companys_daily_return.loc[1,'acc_Rendite'] = ((1+(companys_daily_return.loc[52,'acc_Rendite']*A_KW52_2/(A_KW52_2+A_KWmax_2) + companys_daily_return.loc[last_week2,'acc_Rendite']*A_KWmax_2/(A_KW52_2+A_KWmax_2))/100)*(1 + companys_daily_return.loc[1,'weekly_return']/100)-1)*100
        
        elif i == last_week2 and np.sign(companys_daily_return.loc[last_week2-1,'weekly_return']) == np.sign(companys_daily_return.loc[last_week2,'drei_Wochen_Mittelwert']) == np.sign(companys_daily_return.loc[1,'drei_Wochen_Mittelwert']):
            if companys_daily_return.loc[last_week2-1,'acc_Rendite'] == 0:
                companys_daily_return.loc[last_week2,'acc_Rendite'] = companys_daily_return.loc[last_week2,'weekly_return']
            else:
                companys_daily_return.loc[last_week2,'acc_Rendite'] = ((1+companys_daily_return.loc[last_week2-1,'acc_Rendite']/100) * (1+ companys_daily_return.loc[last_week2,'weekly_return']/100)-1)*100
                
        elif i!=1 and i!=last_week2 and np.sign(companys_daily_return.loc[i,'drei_Wochen_Mittelwert']) == np.sign(companys_daily_return.loc[i-1,'drei_Wochen_Mittelwert']) == np.sign(companys_daily_return.loc[i+1,'drei_Wochen_Mittelwert']):
            if companys_daily_return.loc[i-1,'acc_Rendite'] == 0:
                companys_daily_return.loc[i,'acc_Rendite'] = companys_daily_return.loc[i,'weekly_return']
            else:
                companys_daily_return.loc[i,'acc_Rendite'] =  ((1+companys_daily_return.loc[i-1,'acc_Rendite']/100) *(1+ companys_daily_return.loc[i,'weekly_return']/100)-1)*100
        
        # sollte der Mittelwert nächste Woche von Gewinn zu Verlust oder umgekehrt wechseln,
        # dann soll der Loop die Akkumulierung neu bei Null beginnen, um klar Gewinn bzw. Verlustzeiträume abzugrenzen.
        else:
            companys_daily_return.loc[i,'acc_Rendite'] = 0
    j+=1
companys_daily_return

In [ ]:
#wir füden die EMA zur Kontrolle noch der Tabelle hinzu
companys_daily_return['EMA_year'] = (pd.array(EMA_years)+pd.array(EMA_KW_mthd))/2
companys_daily_return

#### Visualisierung mittels Linien- und Flächendiagramm

In [ ]:
# Leere Figure erstellen
fig2 = go.Figure()
# einen Trace der chronologisch gewichteten wöchentlichen Rendite als Liniendiagramm hinzufügen
fig2.add_trace(go.Scatter(x=companys_daily_return.index,y=companys_daily_return.loc[:,'weekly_return'],
    name="weekly weighted average return  <br>(indicator for volatility)"))
# einen Trace des dreitägigen Mittelwerts hinzufügen
fig2.add_trace(go.Scatter(x=companys_daily_return.index,y=companys_daily_return.loc[:,'EMA_year'],
    name="EMA "+str(jahre)+" years <br>(control of weekly weighted average return)"))
# einen Trace des dreitägigen Mittelwerts hinzufügen
fig2.add_trace(go.Scatter(x=companys_daily_return.index,y=companys_daily_return.loc[:,'drei_Wochen_Mittelwert'],fill='tozeroy',
    name="three weeks average <br>(indicator for +/- return periods)"))
# Zur Renditeeinschätzung von Zeitperioden die in denselben akkumulierte gewichtete wöchentliche Rendite
fig2.add_trace(go.Scatter(x=companys_daily_return.index,y=companys_daily_return.loc[:,'acc_Rendite'],fill='tozeroy',
    name="accumulated return on period <br>(indicator for orders)"))
# Das Layout anpassen
fig2.update_layout(
    title=company+' retrospective weighted Analysis ('+str(jahre) +' years, chronological decay factor = ' + str(rate) +')' +' from '+str(date.today())+ ' ',
    xaxis_title="calendar week",
    yaxis_title="% return",
    font=dict(size=10,color="RebeccaPurple"),
    legend=dict(orientation="h", yanchor="bottom", y=1.02,
                xanchor="right",x=1))
# Die Abstände der Wochen auf der Anzeige der x-Achse für bessere Lesbarkeit verkleinern
fig2.update_xaxes(nticks=30)
fig2.show()

#### Export des Graphen als interaktive .html

In [ ]:
# Abfrage vom Speicher_pfad zum Speicherordner  
file2 = company+'_retrospective_return_analysis_'+ str(jahre) +'_years'

In [ ]:
# Speichern unter dem Speicher_pfad + Datei (per Stringoperationen gebaut)
fig2.write_html(pfad+'/'+file2 +'.html')

## Crossanalyse wöchentlicher Rendite chronologisch abnehmender Gewichtung und Gleichgewichtung

In [ ]:
a = sum(abs(companys_daily_return['drei_Wochen_Mittelwert']))
a_glge = sum(abs(companys_daily_return_glge['drei_Wochen_Mittelwert']))
if a<a_glge:
    companys_daily_return['drei_Wochen_Mittelwert'] = companys_daily_return['drei_Wochen_Mittelwert']*a_glge/a
    companys_daily_return['acc_Rendite'] = companys_daily_return['acc_Rendite']*a_glge/a
else:
    companys_daily_return_glge['drei_Wochen_Mittelwert'] = companys_daily_return_glge['drei_Wochen_Mittelwert']*a/a_glge
    companys_daily_return_glge['acc_Rendite'] = companys_daily_return_glge['acc_Rendite']*a/a_glge

In [ ]:
# relevante Daten für die Crossanalyse zwischen der wöchentlichen Rendite 
# unter chronologisch abnehmender Gewichtung und unter Gleichgewichtung sind:
# der Mittelwert von den beiden drei_Wochen_Mittelwert, sodass ein Kontrollverlauf der Zeiträume akkumuluriender Rendite besteht,
# die beiden acc_Rendite_ge und acc_Rendite_glge bzw. akkumulierenden Renditen
# eine Spalte für die Übereinstimmung der relevanten Daten, und zwar die acc_Rendite_cross akkumulierende Rendite beider Analysen.
cross_weekly_return = pd.DataFrame([companys_daily_return['drei_Wochen_Mittelwert']+companys_daily_return_glge['drei_Wochen_Mittelwert'],companys_daily_return['acc_Rendite'],companys_daily_return_glge['acc_Rendite']]).T
cross_weekly_return.columns = ['drei_Wochen_Mittelwert','acc_Rendite_ge','acc_Rendite_glge']
cross_weekly_return.loc[:,'acc_Rendite_cross']=0
cross_weekly_return.head()

In [ ]:
# Definition erster Bedingung, unter derer Erfüllung acc_Rendite_cross berechnet werden darf:
# sollte die kleinste acc_Rendite um 99% von der größten acc_Rednite im Datensatz entfernt sein, folgt wahr. 
# Hintergrund ist, dass sehr kleine Werte irrelevant für eine Abschätzung der Rendite sind, 
def rel_dist(i):
    quot = min(abs(cross_weekly_return.loc[i,'acc_Rendite_ge']), abs(cross_weekly_return.loc[i,'acc_Rendite_glge'])) / max(abs(cross_weekly_return.loc[i,'acc_Rendite_ge']), abs(cross_weekly_return.loc[i,'acc_Rendite_glge']))
    return quot <0.01

In [ ]:
# 1.Ausschlusskriterium:
# Sollten die beiden acc_Rendite im Datensatz ein unterschiedliches Vorzeichen haben 
# und keiner der beiden im Verhältnis zum anderen sehr klein sein,
# dann lässt sich nicht auf eine Renditeübereinstimmung schließen und diese sollte mit 0 bewertet werden.
# Der folgende Loop ist die Negation obiger Aussage.
for i in range(1,len(cross_weekly_return['acc_Rendite_ge'])):
    if np.sign(cross_weekly_return.loc[i,'acc_Rendite_ge']) == np.sign(cross_weekly_return.loc[i,'acc_Rendite_glge']) or rel_dist(i):
        cross_weekly_return.loc[i,'acc_Rendite_cross']=(cross_weekly_return.loc[i,'acc_Rendite_ge'] + cross_weekly_return.loc[i,'acc_Rendite_glge'])/2

In [ ]:
# 2.Ausschlusskriterium: Die kleinsten Werte, unter 10% des größten Wertes, sollen in der Crossanalyse unberücksichtigt beliben.
# 3.Ausschlusskriterium: Werte mit anderer Tendenz als der drei_Wochen_Mittelwert sollen in der Crossanalyse unberücksichtigt beliben.
for i in range(1,len(cross_weekly_return['acc_Rendite_ge'])):
    if abs(cross_weekly_return.loc[i,'acc_Rendite_cross']) < abs(cross_weekly_return['acc_Rendite_cross'].max())*0.1 or np.sign(cross_weekly_return.loc[i,'acc_Rendite_cross']) != np.sign(cross_weekly_return.loc[i,'drei_Wochen_Mittelwert']):
        cross_weekly_return.loc[i,'acc_Rendite_cross'] = 0

In [ ]:
cross_weekly_return.head()

#### Visualisierung mittels Linien- und Flächendiagramm

In [ ]:
# Leere Figure erstellen
fig3 = go.Figure()
# einen Trace des dreitägigen Mittelwerts hinzufügen zur Kontrolle von Tendenzen
fig3.add_trace(go.Scatter(x=cross_weekly_return.index,y=cross_weekly_return.loc[:,'drei_Wochen_Mittelwert'],
    name="three weeks average <br>(indicator for +/- return periods)"))
# Zur Renditeeinschätzung von Zeitperioden akkumulierte gewichtete wöchentliche Rendite
fig3.add_trace(go.Scatter(x=cross_weekly_return.index,y=cross_weekly_return.loc[:,'acc_Rendite_cross'],fill='tozeroy',
    name="return on period <br>(indicator for orders)"))
# Das Layout anpassen
fig3.update_layout(
    title=company+' Crossanalysis ['+ date_end_titel +' - '+date_start_titel+'] and the last '+ str(jahre) +' years' +' from '+str(date.today())+ ' ',
    xaxis_title="calendar week",
    yaxis_title="% return",
    font=dict(size=10,color="RebeccaPurple"),
    legend=dict(orientation="h", yanchor="bottom", y=1.02,
                xanchor="right",x=1))
# Die Abstände der Wochen auf der Anzeige der x-Achse für bessere Lesbarkeit verkleinern
fig3.update_xaxes(nticks=30)
fig3.show()

In [ ]:
# Abfrage vom Speicher_pfad zum Speicherordner  
file3 = company+'_return_crossanalysis_'+ date_end_titel +'-'+date_start_titel+ '_'+str(jahre) +'_years'

In [ ]:
# Speichern unter dem Speicher_pfad + Datei (per Stringoperationen gebaut)
fig3.write_html(pfad+'/'+file3 +'.html')

#### Export einer Datentabelle der Crossanalyse

In [ ]:
cross_weekly_return = cross_weekly_return.round(2)
cross_weekly_return.reset_index(inplace=True)
cross_weekly_return.columns = ['week',company + ' average return cross %',company + ' acc return weighted ' +str(jahre) +' years %',company + ' acc return '+ date_end_titel +'-'+date_start_titel +' %' , company + ' return cross on period %']
cross_weekly_return

In [ ]:
cross_weekly_return.to_excel(pfad+'/'+file3 +'.xlsx', index=False)

In [ ]:
cross_weekly_return.to_html(pfad+'/'+file3 +'_table.html')